RCSB PDB에서 다운받은 cif 파일을 pdb 파일로 변환

In [2]:
import warnings
import subprocess
from pathlib import Path

import nglview as nv
from openbabel import pybel
from opencadd.structure.core import Structure
import MDAnalysis as mda

import pandas as pd

In [1]:
# cif to pdb
from pathlib import Path
from typing import Union
import gemmi

def cif_to_pdb(cif_path: Union[str, Path], pdb_path: Union[str, Path]) -> None:
    cif_path = Path(cif_path)
    pdb_path = Path(pdb_path)
    pdb_path.parent.mkdir(parents=True, exist_ok=True)

    doc = gemmi.cif.read_file(str(cif_path))
    block = doc.sole_block()
    structure = gemmi.make_structure_from_block(block)
    structure.write_pdb(str(pdb_path))
    print(f"✅ {cif_path} → {pdb_path}")

def convert_recursive(root: Union[str, Path],
                      suffix: str = ".pdb") -> None:
    root = Path(root).resolve()

    # 폴더 아래 모든 하위폴더 포함해 *.cif 파일 전체 검색
    cif_files = sorted(root.rglob("*.cif"))

    if not cif_files:
        print("⚠️ CIF 파일을 찾을 수 없습니다.")
        return

    for cif in cif_files:
        # 동일 폴더에 PDB 생성
        out_path = cif.parent / (cif.stem + suffix)
        try:
            cif_to_pdb(cif, out_path)
        except Exception as e:
            print(f"❌ 변환 실패: {cif} ({e})")


convert_recursive("/home/yongsu/eupatilin/candidate_3")

✅ /home/yongsu/eupatilin/candidate_3/JNK1/3O17/3O17.cif → /home/yongsu/eupatilin/candidate_3/JNK1/3O17/3O17.pdb
✅ /home/yongsu/eupatilin/candidate_3/JNK1/3O2M/3O2M.cif → /home/yongsu/eupatilin/candidate_3/JNK1/3O2M/3O2M.pdb
✅ /home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO.cif → /home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO.pdb


Chain 분리

In [ ]:
awk 'BEGIN{OFS=""}
  /^ATOM|^HETATM/ { if (substr($0,22,1)=="A") print; next }
  /^REMARK|^TER|^END|^MODEL|^ENDMDL/ { print }
' /home/jeongin/eupatilin/data/protein/JNK2/pdb/3NPC.pdb > /home/jeongin/eupatilin/data/protein/JNK2/split/3NPC_protein.pdb

단백질 / 리간드 분리

In [5]:
# split protein - ligand from pdb
from pathlib import Path
import re

IN_PDB  = Path("/home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO.pdb")
OUT_DIR = Path("/home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# ===== 설정 =====
REMOVE_WATER = True        # 물, 이온, 버퍼 제거 여부
REMOVE_IONS = True
REMOVE_BUFFERS = True
KEEP_COMMON_COFAC = False        # HEM, FAD, NAG 등 보조인자도 제거하려면 False
PEPTIDE_LIGAND_MAX_RES = 5       # 이 이하 길이의 별도 체인은 펩타이드형 리간드로 간주
POCKET_CENTER = None             # (x,y,z) 주면 해당 중심 10Å 이내 HET에 +가중치
POCKET_RADIUS = 10.0

WATER = {"HOH", "WAT"}
IONS  = {
    "NA","K","CL","MG","CA","MN","ZN","CU","FE","CO","NI","CD","SR","CS","RB",
    "F","BR","I","BA","AL","AG","HG","PT","AU","GA","PB","LI","SO4"
}
BUFFERS = {"MES","HEPES","TRIS","BIC","CHES","MOPS","ACET","CIT","GOL","MPD","PEG","PGE","EG","EDO","DMS","ISO"}
COMMON_COFAC = {"HEM","HEC","NAD","NADP","FAD","FADH","FMN","COA","PLP","SAM","SAH","NAG","MAN","BGC","POP"}
AA3 = {
    "ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE",
    "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL",
}
PTM_AA = {
    "MSE","SEP","TPO","PTR","YCM","CSO","CSD","CME"
}
NUC = {"A","C","G","T","U","DA","DC","DG","DT","DU","ADE","CYT","GUA","THY","URI"}

def parse_atom_line(line):
    # PDB fixed columns
    rec = line[0:6].strip()
    if rec not in ("ATOM","HETATM"):
        return None
    name = line[12:16]
    resn = line[17:20].strip().upper()
    chain= line[21].strip()
    resi = int(line[22:26])
    icode= line[26].strip()
    x = float(line[30:38]); y = float(line[38:46]); z = float(line[46:54])
    elem = line[76:78].strip().upper()
    return {
        "record":rec,"name":name,"resn":resn,"chain":chain,"resi":resi,"icode":icode,
        "x":x,"y":y,"z":z,"elem":elem,"line":line
    }

def is_heavy(atom):
    # 수소 제외
    return atom["elem"] != "H"

def dist2(a,b):
    return (a["x"]-b[0])**2 + (a["y"]-b[1])**2 + (a["z"]-b[2])**2

# LINK 레코드에서 리간드 단서 수집
link_pairs = []  # (resn, chain, resi)
with IN_PDB.open("r", errors="ignore") as f:
    for line in f:
        if line.startswith("LINK"):
            # 17-20 resName1, 22 chain1, 23-26 seq1 ; 47-50 resName2, 52 chain2, 53-56 seq2
            resn1 = line[17:20].strip().upper()
            ch1   = line[21].strip()
            try:
                i1 = int(line[22:26])
            except:
                continue
            resn2 = line[47:50].strip().upper()
            ch2   = line[51].strip()
            try:
                i2 = int(line[52:56])
            except:
                continue
            link_pairs.append((resn1,ch1,i1))
            link_pairs.append((resn2,ch2,i2))

# ATOM/HETATM 모으기
atoms = []
with IN_PDB.open("r", errors="ignore") as f:
    for line in f:
        rec = line[0:6].strip()
        if rec in ("ATOM","HETATM"):
            p = parse_atom_line(line)
            if p: atoms.append(p)

# 체인 길이(펩타이드형 리간드 감지용)
chain_res_counts = {}
for a in atoms:
    if a["record"] != "ATOM":
        continue
    key = (a["chain"])
    chain_res_counts.setdefault(key, set()).add((a["resn"],a["resi"],a["icode"]))
chain_lengths = {ch: len(resset) for ch,resset in chain_res_counts.items()}

# HET 그룹을 residue 단위로 묶기
from collections import defaultdict
het_groups = defaultdict(list)  # key=(resn, chain, resi, icode)
atom_by_res = defaultdict(list)
for a in atoms:
    key = (a["resn"], a["chain"], a["resi"], a["icode"])
    atom_by_res[key].append(a)
    resn = a["resn"]

    if a["record"] == "HETATM" and resn not in PTM_AA:
        het_groups[key].append(a)

# 후보 리간드 필터링
def is_solvent_like(resn):
    if resn in WATER: return True
    if resn in IONS: return True
    if resn in BUFFERS: return True
    return False

def is_common_cofac(resn):
    return resn in COMMON_COFAC

candidates = []
for key, group in het_groups.items():
    resn, ch, resi, icode = key
    if REMOVE_WATER and resn in WATER:
        continue
    if is_solvent_like(resn):
        continue
    if not KEEP_COMMON_COFAC and is_common_cofac(resn):
        # 보조인자까지 제거하려는 경우 제외
        continue
    heavy_count = sum(1 for a in group if is_heavy(a))
    # 포켓 중심 가중치
    bonus = 0.0
    if POCKET_CENTER and heavy_count > 0:
        # 그룹 원자 중 중심과 가장 가까운 거리 사용
        d2min = min(dist2(a, POCKET_CENTER) for a in group if is_heavy(a))
        if d2min <= POCKET_RADIUS**2:
            bonus += 100.0  # 중심 근처면 강하게 가산
    # LINK 가산점
    if (key[0], key[1], key[2]) in {(r,c,i) for (r,c,i) in link_pairs}:
        bonus += 50.0
    score = heavy_count + bonus
    candidates.append((score, heavy_count, key, group))

# 펩타이드형 리간드 후보(짧은 체인 전체)
peptide_candidates = []
for ch, nres in chain_lengths.items():
    if 0 < nres <= PEPTIDE_LIGAND_MAX_RES:
        # 이 체인 전체를 리간드로 간주
        grp = [a for a in atoms if a["chain"]==ch]
        heavy_count = sum(1 for a in grp if is_heavy(a))
        score = heavy_count + 75.0  # HET보다 덜하지만 LINK보다 높게
        peptide_candidates.append((score, heavy_count, ("PEPTIDE", ch, -1, ""), grp))

# 최종 후보 선택
all_candidates = candidates + peptide_candidates
if not all_candidates:
    raise SystemExit("리간드 후보(HET/짧은 체인)를 찾지 못했습니다. 블랙리스트를 완화해보세요.")

all_candidates.sort(key=lambda x: (x[0], x[1]))  # score, then heavy_count
best = all_candidates[-1]
_, _, best_key, best_group = best

# 리간드/수용체 라인 분리
ligand_set = set(id(a) for a in best_group)
lig_lines, rec_lines = [], []
for a in atoms:
    if id(a) in ligand_set:
        lig_lines.append(a["line"])
    else:
        # 필요시 물, 이온, 버퍼 제거
        if REMOVE_WATER and a["resn"] in WATER:
            continue
        if REMOVE_IONS and a["resn"] in IONS:
            continue
        if REMOVE_BUFFERS and a["resn"] in BUFFERS:
            continue
        rec_lines.append(a["line"])

# 헤더/TER/END 등은 대충 가져와도 무방
other_lines = []
with IN_PDB.open("r", errors="ignore") as f:
    for line in f:
        if not (line.startswith("ATOM") or line.startswith("HETATM")):
            # HEADER, TITLE, REMARK, CRYST1, TER, END 등
            other_lines.append(line)

REC_PDB = OUT_DIR / (IN_PDB.stem + "_protein.pdb")
LIG_PDB = OUT_DIR / (IN_PDB.stem + "_ligand.pdb")

# 간단히: 좌표 레코드만 재구성 + 끝에 END
REC_PDB.write_text("".join(rec_lines) + "END\n")
LIG_PDB.write_text("".join(lig_lines) + "END\n")

print("✅ 자동 분리 완료")
print(" - protein:", REC_PDB)
print(" - ligand  :", LIG_PDB)
print("선택된 리간드 키:", best_key)

✅ 자동 분리 완료
 - protein: /home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO_protein.pdb
 - ligand  : /home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO_ligand.pdb
선택된 리간드 키: ('LBE', 'A', 402, '')


In [3]:
# 6SLG 직접 분리
import gemmi
from pathlib import Path

# ===== 기본 설정 =====
base_dir = Path("/home/yongsu/eupatilin/candidate_2/ERK2/6SLG")

standard_aas = {
    "ALA", "ARG", "ASN", "ASP", "CYS",
    "GLU", "GLN", "GLY", "HIS", "ILE",
    "LEU", "LYS", "MET", "PHE", "PRO",
    "SER", "THR", "TRP", "TYR", "VAL"
}

modified_residues = {
    "MSE", "TPO", "SEP", "PTR", "YCM", "CSO", "CME", "MLY",
    "CSX", "CSD", "HIP", "HID", "HIE", "HYP"
}

# 완전히 제거할 것들 (어느 파일에도 안 들어감)

# 물/용매
solvent_like = {"HOH", "WAT", "DOD", "SOL", "TIP3"}

# 이온들
ion_like = {
    "NA", "K", "CL", "BR", "I",
    "CA", "MG", "MN", "FE", "FE2", "ZN", "CU", "CO",
    "NI", "CD", "HG", "SR", "CS", "LI", "BA", "YB"
}

# 🔹 버퍼/첨가제(통째로 제거하고 싶은 것들)
buffer_like = {
    # 버퍼 계열
    "HEP", "EPE",  # HEPES
    "TRS",         # TRIS
    "MES",         # MES
    "PIP",         # PIPES
    "MPO",         # MOPS
    "ADA",         # ADA 등 필요시 추가

    # 소형 유기 분자 (보통 cryo/결정화 첨가제)
    "GOL",         # glycerol
    "PEG", "PG4", "PG5", "PG6",  # PEG 계열
    "ACT", "ACE",  # acetate
    "CIT",         # citrate
    "FMT",         # formate
    "SO4", "PO4",  # sulfate/phosphate (완전히 버리고 싶으면)
    "EOH",         # ethanol
    "IPA",         # isopropanol
    "DMS",         # DMSO
    "EDO"
}

# 여기서 전부 합쳐서 "완전히 제거" 그룹으로 사용
exclude_residues = solvent_like | ion_like | buffer_like


# ===== PDB 파일 순회 =====
for pdb_file in base_dir.rglob("*.pdb"):
    if pdb_file.name.endswith("_protein.pdb") or pdb_file.name.endswith("_ligand.pdb"):
        continue

    try:
        print(f"[INFO] 처리 중: {pdb_file}")

        # Gemmi 구조 읽기
        st = gemmi.read_structure(str(pdb_file))

        prot_st = gemmi.Structure()
        lig_st = gemmi.Structure()
        prot_st.name = st.name
        lig_st.name = st.name

        parsed_residues = set()  # Gemmi에서 처리된 residue 기록

        # --- ① Gemmi 계층 기반 탐색 ---
        for model in st:
            prot_model = gemmi.Model(model.name)
            lig_model = gemmi.Model(model.name)
            for chain in model:
                chain_name = chain.name.strip() if chain.name.strip() else "_"
                prot_chain = gemmi.Chain(chain_name)
                lig_chain = gemmi.Chain(chain_name)
                for res in chain:
                    resname = res.name.strip().upper()
                    parsed_residues.add(resname)
                    if resname in standard_aas or resname in modified_residues:
                        prot_chain.add_residue(res)
                    elif resname not in exclude_residues:
                        lig_chain.add_residue(res)
                if len(prot_chain):
                    prot_model.add_chain(prot_chain)
                if len(lig_chain):
                    lig_model.add_chain(lig_chain)
            if len(prot_model):
                prot_st.add_model(prot_model)
            if len(lig_model):
                lig_st.add_model(lig_model)

        # --- ② line-based 보충 탐색 (계층에 없는 residue 보완) ---
        unparsed_lines = []
        with open(pdb_file) as f:
            for line in f:
                if not line.startswith(("ATOM", "HETATM")):
                    continue
                resname = line[17:20].strip().upper()
                if resname not in parsed_residues:
                    unparsed_lines.append(line)

        if unparsed_lines:
            print(f"[WARN] 계층 구조에서 누락된 원자 {len(unparsed_lines)}개 → line 기반으로 보완")
            for line in unparsed_lines:
                resname = line[17:20].strip().upper()
                if resname in standard_aas or resname in modified_residues:
                    # 단백질 보충
                    model = prot_st[0] if len(prot_st) > 0 else prot_st.add_model("0")
                    chain = model.find_chain("_") or model.add_chain("_")
                    residue = gemmi.make_residue("UNK", 0, ' ')
                    residue.name = resname
                    atom = gemmi.make_small_atom(line[12:16].strip(), float(line[30:38]), float(line[38:46]), float(line[46:54]))
                    residue.add_atom(atom)
                    chain.add_residue(residue)
                elif resname not in exclude_residues:
                    # 리간드 보충
                    model = lig_st[0] if len(lig_st) > 0 else lig_st.add_model("0")
                    chain = model.find_chain("_") or model.add_chain("_")
                    residue = gemmi.make_residue("UNK", 0, ' ')
                    residue.name = resname
                    atom = gemmi.make_small_atom(line[12:16].strip(), float(line[30:38]), float(line[38:46]), float(line[46:54]))
                    residue.add_atom(atom)
                    chain.add_residue(residue)

        # --- ③ 결과 저장 ---
        protein_out = pdb_file.parent / f"{pdb_file.stem}_protein.pdb"
        ligand_out  = pdb_file.parent / f"{pdb_file.stem}_ligand.pdb"

        if len(prot_st) > 0:
            prot_st.write_minimal_pdb(str(protein_out))
            print(f"✅ 단백질 저장 완료: {protein_out}")
        if len(lig_st) > 0:
            lig_st.write_minimal_pdb(str(ligand_out))
            print(f"✅ 리간드 저장 완료: {ligand_out}")

        print()

    except Exception as e:
        print(f"❌ 오류 발생 ({pdb_file.name}): {e}")




[INFO] 처리 중: /home/yongsu/eupatilin/candidate_2/ERK2/6SLG/6SLG.pdb
✅ 단백질 저장 완료: /home/yongsu/eupatilin/candidate_2/ERK2/6SLG/6SLG_protein.pdb
✅ 리간드 저장 완료: /home/yongsu/eupatilin/candidate_2/ERK2/6SLG/6SLG_ligand.pdb



대체원자 포켓과의 거리

In [18]:
from pathlib import Path
import csv
import math
import gemmi

# ===== 입력 설정 =====
PROTEIN_PDB = Path("/home/yongsu/eupatilin/candidate_2/ERK2/6SLG/6SLG_protein.pdb")
POCKET_CSV  = Path("/home/yongsu/eupatilin/candidate_2/ERK2/6SLG/6SLG_pocket_info.csv")

OCC_THRESHOLD = 0.999   # 점유율 < 이 값 또는 altloc 있으면 대체원자
DIST_CUTOFF   = 4.0     # 대체원자-포켓 atom 거리 cutoff (Å)


def read_pocket_from_csv(csv_path: Path):
    """pocket_info.csv에서 center/size 읽기"""
    with csv_path.open() as f:
        reader = csv.DictReader(f)
        row = next(reader)
    center = (
        float(row["center_x"]),
        float(row["center_y"]),
        float(row["center_z"]),
    )
    size = (
        float(row["size_x"]),
        float(row["size_y"]),
        float(row["size_z"]),
    )
    return center, size


def is_in_box(pos: gemmi.Position, center, size) -> bool:
    """점이 박스 내부에 있는지 검사"""
    cx, cy, cz = center
    sx, sy, sz = size
    hx, hy, hz = sx / 2.0, sy / 2.0, sz / 2.0
    return (
        (cx - hx) <= pos.x <= (cx + hx) and
        (cy - hy) <= pos.y <= (cy + hy) and
        (cz - hz) <= pos.z <= (cz + hz)
    )


# ===== 1) 포켓 박스 읽기 =====
center, size = read_pocket_from_csv(POCKET_CSV)

# ===== 2) 단백질 구조 읽기 =====
prot_st = gemmi.read_structure(str(PROTEIN_PDB))
prot_model = prot_st[0]

# ===== 3) 포켓 atom (박스 안 단백질 atom) =====
pocket_atoms = []  # (chain_name, residue_obj, atom_obj)

for chain in prot_model:
    for res in chain:
        for atom in res:
            if is_in_box(atom.pos, center, size):
                pocket_atoms.append((chain.name, res, atom))

# ===== 4) 단백질 전체에서 대체원자 찾기 =====
sub_atoms = []  # (chain_name, residue_obj, atom_obj)

for chain in prot_model:
    for res in chain:
        for atom in res:
            # has_altloc() 있으면 그걸 사용, 없으면 altloc 값으로 판단
            has_altloc = getattr(atom, "has_altloc", lambda: (atom.altloc != '\0'))()
            if atom.occ < OCC_THRESHOLD or has_altloc:
                sub_atoms.append((chain.name, res, atom))

total_sub_atoms = len(sub_atoms)

# ===== 5) 포켓과 4 Å 이내에 있는 대체원자 수 + 최소 거리 계산 =====
# 같은 자리(체인, residue, atom 이름)는 하나로 보고, 그 자리의 최소 거리만 저장
contacting_sites = {}  # key: (chain, resid_num, resname, atomname) -> min_distance

for c_s, r_s, a_s in sub_atoms:
    min_d = float("inf")
    for c_p, r_p, a_p in pocket_atoms:
        d = a_s.pos.dist(a_p.pos)
        if d < min_d:
            min_d = d
    if min_d <= DIST_CUTOFF:
        key = (c_s, r_s.seqid.num, r_s.name, a_s.name.strip())
        # 이미 있으면 더 작은 거리로 업데이트
        if key in contacting_sites:
            contacting_sites[key] = min(contacting_sites[key], min_d)
        else:
            contacting_sites[key] = min_d

num_contacting = len(contacting_sites)

# ===== 6) 결과 출력 =====
print(f"대체원자(altloc/부분점유) 원자 수: {total_sub_atoms}")
print(f"{DIST_CUTOFF:.1f} Å 이내에서 포켓과 상호작용하는 대체원자 수: {num_contacting}")

if num_contacting == 0:
    print(f"→ 포켓과 {DIST_CUTOFF:.1f} Å 이내에 있는 대체원자가 없습니다.")
else:
    print(f"\n{DIST_CUTOFF:.1f} Å 이내 대체원자 목록 (이름 + 최소 거리):")
    for (chain_name, resid_num, resname, atom_name), d in sorted(
        contacting_sites.items(), key=lambda x: x[1]
    ):
        print(
            f"- chain {chain_name}, res {resname} {resid_num}, "
            f"atom {atom_name}: {d:.2f} Å"
        )


대체원자(altloc/부분점유) 원자 수: 64
4.0 Å 이내에서 포켓과 상호작용하는 대체원자 수: 6

4.0 Å 이내 대체원자 목록 (이름 + 최소 거리):
- chain A, res CYS 40, atom N: 0.00 Å
- chain A, res CYS 40, atom CA: 0.00 Å
- chain A, res CYS 40, atom C: 0.00 Å
- chain A, res CYS 40, atom O: 0.00 Å
- chain A, res CYS 40, atom CB: 0.00 Å
- chain A, res CYS 40, atom SG: 0.00 Å


단백질 pdb 파일 pdbqt로 변환

In [ ]:
prepare_receptor4.py -r protein.pdb -o protein.pdbqt

Pocket size 계산

In [6]:
from pathlib import Path
import csv

# ===== 사용자 설정 =====
MODE = "A"  # "A" = 리간드 파일 1개로 계산, "B" = 단백질 PDB에서 리간드 resname 추출
OUT_CSV = Path("/home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO_pocket_info.csv")

# (A) 모드: 리간드 파일 경로 (PDB 또는 PDBQT)
LIGAND_FILE = Path("/home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO_ligand.pdb")  

# (B) 모드: 단백질 PDB + 리간드 resname (3-letter)
PROTEIN_PDB = Path("/home/yongsu/eupatilin/candidate_3/p38_MAPK/6SFO/6SFO_protein.pdb")

# 박스 설정
PAD = 5.0       # 각 축으로 ±PAD Å 패딩 (size = (max-min) + 2*PAD)
MIN_SIZE = 20.0 # 각 축 최소 박스 길이(Å). LDHA 포켓은 18–26Å 권장 → 20Å 기본
MAKE_CUBE = False  # True면 size_x=y=z=세 축 중 최댓값으로 큐브화(옵션)


def read_coords_from_pdb_like(path: str):
    """PDB/PDBQT에서 ATOM/HETATM 좌표 읽기"""
    xs, ys, zs = [], [], []
    with open(path, "r", errors="ignore") as f:
        for line in f:
            if not (line.startswith("ATOM") or line.startswith("HETATM")):
                continue
            try:
                x = float(line[30:38]); y = float(line[38:46]); z = float(line[46:54])
            except ValueError:
                continue
            xs.append(x); ys.append(y); zs.append(z)
    return xs, ys, zs

def bbox_center_size(xs, ys, zs, pad=5.0, min_size=10.0, make_cube=False):
    """좌표의 AABB로 center/size 계산"""
    if not xs:
        raise ValueError("좌표가 비었습니다. 입력을 확인하세요.")
    minx, maxx = min(xs), max(xs)
    miny, maxy = min(ys), max(ys)
    minz, maxz = min(zs), max(zs)
    cx, cy, cz = (minx+maxx)/2.0, (miny+maxy)/2.0, (minz+maxz)/2.0
    sx = max((maxx-minx) + 2.0*pad, min_size)
    sy = max((maxy-miny) + 2.0*pad, min_size)
    sz = max((maxz-minz) + 2.0*pad, min_size)
    if make_cube:
        m = max(sx, sy, sz)
        sx = sy = sz = m
    return (cx, cy, cz), (sx, sy, sz)


# === 실행 ===
assert LIGAND_FILE.exists(), f"리간드 파일 없음: {LIGAND_FILE}"
xs, ys, zs = read_coords_from_pdb_like(str(LIGAND_FILE))
center, size = bbox_center_size(xs, ys, zs, pad=PAD, min_size=MIN_SIZE, make_cube=MAKE_CUBE)

OUT_CSV.parent.mkdir(parents=True, exist_ok=True)
with OUT_CSV.open("w", newline="") as w:
    writer = csv.DictWriter(w, fieldnames=["center_x","center_y","center_z","size_x","size_y","size_z","pad","min_size","cube"])
    writer.writeheader()
    writer.writerow({
        "center_x": center[0], "center_y": center[1], "center_z": center[2],
        "size_x": size[0], "size_y": size[1], "size_z": size[2],
        "pad": PAD, "min_size": MIN_SIZE, "cube": int(MAKE_CUBE)
    })

print("✅ pocket_info.csv 저장 완료")
print(f"center=({center[0]:.3f},{center[1]:.3f},{center[2]:.3f})  size=({size[0]:.3f},{size[1]:.3f},{size[2]:.3f})")

✅ pocket_info.csv 저장 완료
center=(0.050,1.071,-19.152)  size=(20.423,20.000,25.774)


Pocket size 파일에 통합

In [7]:
import pandas as pd
import re
import os
import sys
from pathlib import Path

OUT_DIR = Path("/home/yongsu/eupatilin/candidate_3")
POCKET_DIR = Path("/home/yongsu/eupatilin/candidate_3")

def build_pocket_master(pocket_dir: Path, out_csv: Path):
    rows = []
    for csv in pocket_dir.rglob("*pock*et_info.csv"):
        m = re.match(r"([0-9A-Za-z]{4})(?:_[A-Za-z0-9]+)?_pock.*et_info\.csv$", csv.name)
        pdb_id = m.group(1) if m else None
        if not pdb_id: 
            continue
        df = pd.read_csv(csv)
        if not set(["center_x","center_y","center_z","size_x","size_y","size_z"]).issubset(df.columns):
            print(f"[WARN] 컬럼 부족: {csv.name}")
            continue
        r = df.iloc[0]  # 구조당 1행 가정
        rows.append({
            "pdb_id": pdb_id,
            "center_x": r["center_x"], "center_y": r["center_y"], "center_z": r["center_z"],
            "size_x": r["size_x"], "size_y": r["size_y"], "size_z": r["size_z"]
        })
    master = pd.DataFrame(rows).drop_duplicates("pdb_id")
    master.to_csv(out_csv, index=False)
    return master

POCKET_MASTER = OUT_DIR / "ALL_pocket_info.csv"
master = build_pocket_master(POCKET_DIR, POCKET_MASTER)
print(f"[OK] pocket_master 생성: {POCKET_MASTER} (행 {len(master)})")

[OK] pocket_master 생성: /home/yongsu/eupatilin/candidate_3/ALL_pocket_info.csv (행 2)


Docking(docking_process.py 실행)

In [ ]:
python 파일경로.py


Docked 파일에서 affinity 파싱

In [8]:
#!/usr/bin/env python3
from pathlib import Path
import pandas as pd

DOCK_DIR = Path("/home/yongsu/eupatilin/candidate_3")

def parse_affinities_from_pdbqt(pose_path: Path):
    """
    smina/ vina 결과 pdbqt에서
    REMARK VINA RESULT 줄들을 모두 읽어서
    (pose_index, affinity) 리스트로 반환.
    """
    affinities = []
    pose_idx = 1

    with open(pose_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if "minimizedAffinity" in line:
                parts = line.strip().split()
                aff = None
                for tok in reversed(parts):
                    try:
                        aff = float(tok)
                        break
                    except ValueError:
                        continue
                if aff is not None:
                    affinities.append((pose_idx, aff))
                    pose_idx += 1
    return affinities

def main():
    rows = []

    # *_docked.pdbqt 파일들 전부 순회
    for pose_path in sorted(DOCK_DIR.rglob("*_docked.pdbqt")):
        # 파일명에서 pdb_id 추출: 예) 3E7O_docked.pdbqt → 3E7O
        pdb_id = pose_path.name.split("_")[0]

        aff_list = parse_affinities_from_pdbqt(pose_path)
        if not aff_list:
            print(f"[WARN] minimizedAffinity 정보 없음: {pose_path.name}")
        for pose_idx, aff in aff_list:
            rows.append({
                "pdb_id": pdb_id,
                "pose_index": pose_idx,   # 1 = best pose
                "affinity": aff,
                "file": str(pose_path),
            })

    if not rows:
        print("추출된 affinity가 없습니다.")
        return

    df = pd.DataFrame(rows)
    out_csv = DOCK_DIR / "docking_affinities.csv"
    df.to_csv(out_csv, index=False)
    print("✓ 저장 완료:", out_csv)

if __name__ == "__main__":
    main()

✓ 저장 완료: /home/yongsu/eupatilin/candidate_3/docking_affinities.csv
